In [4]:
import os
from PIL import Image, PngImagePlugin
import glob
import numpy as np
np.set_printoptions(threshold=25)
import scipy.fftpack
import pandas as pd
import h5py
from collections import deque

In [2]:
def interlace(input_paths, save = False):
    frames = [Image.open(fn).convert("RGB") for fn in input_paths]
    assert len(frames) > 0, "Must have at least one input frame."
    output_name = input_paths[0].split('.')[0] + '_il' + input_paths[0].split('.')[1]
    sizes = set()
    for fr in frames:
        sizes.add(fr.size)
    assert len(sizes) == 1, "All input images must have the same size."
    w, h = sizes.pop()
    N = len(frames)
    output_image = Image.new("RGB", (w, h*N))
    for j in range(h):
        for i in range(w):
              for fn, f in enumerate(frames):
                    output_image.putpixel((i, j*N+fn), f.getpixel((i, j)))
    # When loading this image, the graphics library expects to find a text
    # chunk that specifies how many frames this animation represents.  If
    # you post-process the output of this script with some kind of
    # optimizer tool (eg pngcrush or zopflipng) make sure that your
    # optimizer preserves this text chunk.
    meta = PngImagePlugin.PngInfo()
    meta.add_text("Frames", str(N))
    if save:
        output_image.save(output_name, pnginfo=meta)
    return output_image


In [4]:
def create_interlace_input(frames, window_size = 3):
    frame_window = deque(frames[0:window_size])
    interlace_inputs = []
    for frame in frames[window_size:]:
        frame_window.append(frame)
        frame_window.popleft()
        interlace_input = list(frame_window)
        interlace_inputs.append(interlace_input)
    return interlace_inputs

In [15]:
def get_hash(im, hash_size, high_freq_factor):
    img_size = hash_size * high_freq_factor
    image = im.convert("L").resize((img_size, img_size), Image.ANTIALIAS)
    pixels = np.asarray(image)
    dct = scipy.fftpack.dct(scipy.fftpack.dct(pixels, axis=0), axis=1)
    dctlowfreq = dct[:hash_size, :hash_size]
    med = np.median(dctlowfreq)
    diff = dctlowfreq > med
    phash = [1 if x == True else 0 for x in diff.flatten()]
    return phash

In [6]:
def write_hashes_elementwise_to_hdf5(phashes, name = ''):
    len_hash = len(phashes[0])
    n_frames = len(phashes)
    
    hdf5_store = h5py.File("./phashes{}.hdf5".format('_'+name), "a")
    phashes_hdf5 = hdf5_store.create_dataset("phashes", (n_frames, len_hash), compression="gzip")

    for i in range(0, len(phashes)):
        phashes_hdf5[i] = phashes[i]    

In [12]:
def get_interlashed_hashes(interlace_inputs, hash_size, high_freq_factor):
    phashes = []
    for interlace_input in interlace_inputs:
        output_image = interlace(interlace_input)
        phash = get_hash(output_image, hash_size, high_freq_factor)
        phashes.append(phash)
    return phashes


In [17]:
# input

wd_frame_paths = '/home/emsala/Documenten/Studie/These/phashing/dataset/films/film_1/frames/*'
hash_size = 15
high_freq_factor = 4


In [ ]:
#main

frame_paths = sorted(glob.glob(wd_frame_paths))

interlace_inputs = create_interlace_input(frame_paths)
phashes = get_interlashed_hashes(interlace_inputs, hash_size, high_freq_factor)

write_elementwise_to_hdf5(frame_paths, phashes, name = 'il')